In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# predefined parameter

course_platform = 'futurelearn'
#course_subject = 'infosec'
#course_level = 'advanced'
course_duration = '10%2B'

total_course = 1849

In [3]:
course_link = list()
course_title = list()
course_provider = list()
course_rating = list()
course_nreviewer = list()
course_workload = list()

In [4]:
import numpy as np
import pandas as pd

data = []

max_display = 15

total_page = np.ceil(total_course/max_display).astype(int)

header = {'User-Agent': 'python-requests/2.26.0', 
          'Accept-Encoding': 'gzip, deflate, br', 
          'Accept': '*/*', 'Connection': 'keep-alive'
         }

In [5]:
for current_page in range(1, total_page+1):
    page = requests.get(
        f"https://www.classcentral.com/provider/{course_platform}?duration={course_duration}&page={current_page}", 
        headers = header)
    soup = BeautifulSoup(page.content, 'html.parser')

    for i in range(0, max_display):
        
        try:

            course_link.append(soup.find_all('a', class_="hover-no-underline margin-bottom-xxsmall row vert-align-middle")[i]["href"])
            course_title.append(soup.find_all('h2', class_="text-1 weight-semi line-tight margin-bottom-xxsmall")[i].get_text())
            #course_provider.append(soup.find_all('span', attrs={"style": "line-height: 100%;"})[i].find("a")["href"])
            #course_rating.append(soup.find_all('a', class_="hover-no-underline margin-bottom-xxsmall row vert-align-middle")[i].find("span")["aria-label"])
            #course_nreviewer.append(soup.find_all('span', class_="text-3 color-gray margin-left-xxsmall")[i].get_text())
            #course_workload.append(soup.find_all('span', attrs={"aria-label": "Workload and duration"})[i].get_text())
            
        except IndexError:
            
            None
            
data.append({
    'Link' : course_link,
    'Title' : course_title,
    #'Subject' : course_subject,
    ##'Provider' : course_provider,
    ##'Rating' : course_rating,
    ##'Num_Reviewer' : course_nreviewer,
    ##'Workload' : course_workload
    #'Platform' : course_platform
})

df = pd.DataFrame(data)


In [6]:
df

,Link,Title
0,"[/course/end-of-life-care-16932, /course/disco...","[End of Life Care: Challenges and Innovation, ..."


In [7]:
from itertools import zip_longest

cols = ['Link', 
        'Title'
        #'Provider', 
        #'Rating', 
        #'Num_Reviewer', 
        #'Workload'
       ]
def fun(x):
    return list(zip_longest(*x))

s = pd.Series(map(fun,df[cols].to_records(index=False))).explode()

out = df.drop(cols,1).join(pd.DataFrame(s.tolist(),columns=cols,index=s.index))

C:\Users\NURADH~1\AppData\Local\Temp/ipykernel_17208/2970702368.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  out = df.drop(cols,1).join(pd.DataFrame(s.tolist(),columns=cols,index=s.index))


In [8]:
out['Platform'] = course_platform
#out['subject'] = course_subject
out['Duration'] = course_duration

In [9]:
out

,Link,Title,Platform,Duration
0,/course/end-of-life-care-16932,End of Life Care: Challenges and Innovation,futurelearn,10%2B
0,/course/discover-dentistry-1249,Discover Dentistry,futurelearn,10%2B
0,/course/technical-report-writing-for-engineers...,Technical Report Writing for Engineers,futurelearn,10%2B
0,/course/songwriting-3369,How to Write Your First Song,futurelearn,10%2B
0,/course/mindfulness-life-9078,Maintaining a Mindful Life,futurelearn,10%2B
...,...,...,...,...
0,/course/futurelearn-desigualdades-en-america-l...,Desigualdades en América Latina y el Caribe: I...,futurelearn,10%2B
0,/course/futurelearn-copyright-historia-cultura...,"Copyright: Historia, Cultura, Industria",futurelearn,10%2B
0,/course/futurelearn-copyright-historia-cultura...,"Copyright: História, Cultura, Indústria",futurelearn,10%2B
0,/course/revolutionising-the-food-chain-spanish...,Revolucionar la cadena alimentaria mediante la...,futurelearn,10%2B


In [10]:
out.to_csv(course_platform + "_duration_" + course_duration + ".csv", index=False)